In [ ]:
import json
import psycopg2

_db='fred'
_user='fred'
_psw='763160'
_host='localhost'
_port=5432


In [ ]:
def connect():
    con = None
    try:
        con = psycopg2.connect(database=_db,
                               user=_user,
                               password=_psw,
                               host=_host,
                               port=_port)

    except psycopg2.DatabaseError, e:
        print('Error %s' % e)

    return con

In [ ]:
def delete_table(table):
    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute('DROP TABLE ' + table)
            con.commit()
        except Exception as e:
            return False

        con.close()
        return True
    return False

In [ ]:
def create_table(table):
    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute('CREATE TABLE ' + table + '(ID INT PRIMARY KEY NOT NULL, data TEXT NOT NULL)')
            con.commit()
        except Exception as e:
            return False

        con.close()
        return True
    return False

print(create_table('test'))
print(create_table('test'))
print(delete_table('test'))

In [ ]:
def table_exist(table):
    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute("SELECT * FROM information_schema.tables WHERE table_name='" + table + "';")
        except Exception as e:
            return False

        con.close()
        return bool(cur.rowcount)
    return False

create_table('test')
print(table_exist('toto'))
print(table_exist('test'))
delete_table('test')

In [ ]:
def create_row(table, id, data):
    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute("INSERT INTO " + table + "(ID,data) " + "VALUES("+ str(id) + ", '"+ data + "')" )
            con.commit()
        except Exception as e:
            return False

        con.close()
        return True
    return False

create_table('test')
print(create_row('test', 1, 'toto'))
print(create_row('test', 1, 'test'))
delete_table('test')

In [ ]:
def row_exist(table, id):
    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute("SELECT * FROM " + table + " WHERE ID='" + str(id) + "';")
        except Exception as e:
            return False

        con.close()
        return bool(cur.rowcount)
    return False

In [ ]:
def store(table, id, data):
    if not table_exist(table):
        create_table(table)
    if not row_exist(table, id):
        create_row(table, id, data)

    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute('UPDATE ' + table + ' SET data=\''+ json.dumps(data) +'\' WHERE ID=' + str(id))
            con.commit()
        except Exception as e:
            return False

        con.close()
        return True
    return False

print(store('test', 1, 'toto'))
print(store('test', 1, 'test'))
delete_table('test')

In [ ]:
def restore(table, id):
    if not table_exist(table) or not row_exist(table, id):
        return ''

    con = connect()
    if con :
        try:
            cur = con.cursor()
            cur.execute('SELECT data FROM ' + table + ' WHERE ID=' + str(id))
            records = cur.fetchall()
            data = json.loads(records[0][0])
        except Exception as e:
            return None

        con.close()
        return data
    return None
store('test', 1, 'toto')
print(restore('test2', 1))
print(restore('test', 2))
print(restore('test', 1))
delete_table('test')

In [1]:
from postgres_store import postgres_store

In [2]:
db = postgres_store('fred', 'fred', '763160', 'localhost', 5432)

In [3]:
db.store('users', 1, [{'name': 'test'}])
db.restore('users', 1)

[{u'name': u'test'}]